<a href="https://colab.research.google.com/github/nachoacev/practice-data-science/blob/main/IntroSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro to SQL

Structured Query Language, or **SQL**, is the programming language used with databases, and it is an important skill for any data scientist. We'll build our SQL skills using **BigQuery**, *a web service that lets you apply SQL to huge datasets*.

We'll learn the basics of accessing and examining BigQuery datasets.

# BigQuery Commands `Client`, `dataset()`, `get_dataset()`

To use BigQuery, we'll import the Python package below

## Install and Authenticate

In [2]:
# Install BigQuery library
#!pip install google-cloud-bigquery

In [3]:
# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
from google.cloud import bigquery

## Set up the BigQuery client

The first step in the workflow is to create a `Client` object. As we'll soon see, this `Client` object will play a central role in retrieving information from BigQuery datasets.

In [5]:
# Create a "Client" object
client = bigquery.Client()

We'll work with a dataset of posts on Hacker News, a website focusing on computer science and cybersecurity news.

In BigQuery, each dataset is contained in a corresponding project. In this case, our `hacker_news` dataset is contained in the `bigquery-public-data` project. To access the dataset,

- We begin by constructing a reference to the dataset with the `dataset()` method.
- Next, we use the `get_dataset()` method, along with the reference we just constructed, to fetch the dataset.

In [6]:
# Construct a reference to the "openaq" dataset
dataset_ref = client.dataset("openaq", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [7]:
# Construct a reference to the "chicago_crime" dataset
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Every dataset is just a collection of tables. You can think of a dataset as a spreadsheet file containing multiple tables, all composed of rows and columns.

We use the `list_tables()` method to list the tables in the dataset.

In [9]:
# List all the tables in "chicago_crime" dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset
for table in tables:
  print(table.table_id)

crime


Similar to how we fetched a dataset, we can fetch a table.

In [10]:
# Construct a reference to the "crime" table
table_ref = dataset_ref.table("crime")

# API request - fetch the table
table = client.get_table(table_ref)

# Table Squema `SchemaField`, `list_rows()`,

The structure of a table is called its **schema**. We need to understand a table's schema to effectively pull out the data we want.

In this example, we'll investigate the `crime` table that we fetched above.

In [11]:
# Print information on all columns in the "crime" table in the "chicago_crime" dataset
table.schema

[SchemaField('unique_key', 'INTEGER', 'REQUIRED', None, None, (), None),
 SchemaField('case_number', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, None, (), None),
 SchemaField('block', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('iucr', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('primary_type', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('description', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('location_description', 'STRING', 'NULLABLE', None, None, (), None),
 SchemaField('arrest', 'BOOLEAN', 'NULLABLE', None, None, (), None),
 SchemaField('domestic', 'BOOLEAN', 'NULLABLE', None, None, (), None),
 SchemaField('beat', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('district', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('ward', 'INTEGER', 'NULLABLE', None, None, (), None),
 SchemaField('community_area', 'INTEGER', 'NULLABLE', None, None, (), None),
 

Each `SchemaField` tells us about a specific column (which we also refer to as a **field**). In order, the information is:

- The **name** of the column
- The **field type** (or datatype) in the column
- The **mode** of the column ('NULLABLE' means that a column allows NULL values, and is the default)
- A **description** of the data in that column

We can use the `list_rows()` method to check just the first five lines of of the full table to make sure this is right. (Sometimes databases have outdated descriptions, so it's good to check.) This returns a BigQuery `RowIterator` object that can quickly be converted to a pandas DataFrame with the `to_dataframe()` method.

In [12]:
# Preview the first five lines of the "full" table
client.list_rows(table, max_results=5).to_dataframe()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,4954348,HM567052,2006-08-28 11:05:00+00:00,006XX W RANDOLPH ST,031A,ROBBERY,ARMED: HANDGUN,PARKING LOT/GARAGE(NON.RESID.),False,False,...,42,28,03,1172203.0,1901233.0,2006,2018-02-10 15:50:01+00:00,41.884432,-87.643103,"(41.884432451, -87.643102964)"
1,3255033,HK280478,2004-04-03 16:24:00+00:00,005XX W MADISON ST,031A,ROBBERY,ARMED: HANDGUN,ALLEY,False,False,...,42,28,03,1173039.0,1900311.0,2004,2018-02-28 15:56:25+00:00,41.881884,-87.640060,"(41.881883933, -87.640060441)"
2,12859590,JF435025,2022-10-15 01:04:00+00:00,0000X E WACKER DR,031A,ROBBERY,ARMED - HANDGUN,CONVENIENCE STORE,False,False,...,42,32,03,1176621.0,1902155.0,2022,2023-01-03 15:40:27+00:00,41.886864,-87.626852,"(41.886863814, -87.626851797)"
3,12515419,JE413487,2021-10-18 08:34:00+00:00,0000X E WACKER DR,031A,ROBBERY,ARMED - HANDGUN,CONVENIENCE STORE,False,False,...,42,32,03,1176621.0,1902155.0,2021,2021-10-25 15:42:49+00:00,41.886864,-87.626852,"(41.886863814, -87.626851797)"
4,6757567,HR170212,2009-02-14 12:40:00+00:00,002XX S CANAL ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,...,2,28,03,1173177.0,1899236.0,2009,2018-02-28 15:56:25+00:00,41.878931,-87.639586,"(41.878931004, -87.639585621)"


The `list_rows()` method will also let us look at just the information in a specific column. If we want to see the first five entries in the `case_number` column, for example, we can do that

In [23]:
# Preview first five entries in the "case_number" column of the table "crime"
client.list_rows(table, selected_fields=table.schema[1:2], max_results=5).to_dataframe()

,case_number
0,HM567052
1,HK280478
2,JF435025
3,JE413487
4,HR170212


SchemaField('date', 'TIMESTAMP', 'NULLABLE', None, None, (), None)